In [65]:
import pandas as pd
import matplotlib.pyplot as plt

from urllib.request import urlopen

import json

In [11]:
_5min = pd.read_csv('G:/내 드라이브/취업/Contest/2307_PublicData/data/한국전력거래소_5분단위 전력수급현황_20220407.csv', encoding='euc-kr')
solar = pd.read_csv('G:/내 드라이브/취업/Contest/2307_PublicData/data/한국전력거래소_지역별 시간별 태양광 발전량_20230228.csv', encoding='euc-kr')

In [52]:
# plt.plot(_5min['현재수요(MW)'])
_5min

,기준일시,공급능력(MW),현재수요(MW),최대예측수요(MW),공급예비력(MW),공급예비율(퍼센트),운영예비력(MW),운영예비율(퍼센트)
0,2012-06-01 00:00,64355.0,51726.33,59400,12628.67,24.41439,12628.67,24.41439
1,2012-06-01 00:05,64355.0,51342.53,60800,13012.47,25.34442,13012.47,25.34442
2,2012-06-01 00:10,64355.0,51583.17,60800,12771.83,24.75969,12771.83,24.75969
3,2012-06-01 00:15,64355.0,51631.69,60800,12723.31,24.64245,12723.31,24.64245
4,2012-06-01 00:20,64355.0,51252.66,60800,13102.35,25.56423,13102.35,25.56423
...,...,...,...,...,...,...,...,...
1035062,2022-04-07 23:35,82791.9,63877.30,69300,18914.70,29.61100,10803.00,16.91210
1035063,2022-04-07 23:40,82768.1,63525.30,69300,19242.80,30.29160,10696.40,16.83800
1035064,2022-04-07 23:45,82767.9,63506.20,69300,19261.70,30.33040,10501.30,16.53580
1035065,2022-04-07 23:50,82762.4,62819.60,69300,19942.80,31.74610,10706.30,17.04290


In [30]:
# (_5min.isnull().sum()/ _5min.shape[0]).sort_values(ascending=False)
(solar.isnull().sum()/ solar.shape[0]).sort_values(ascending=False)

 풍력 발전량(MWh)      0.352941
거래일자              0.000000
거래시간              0.000000
지역                0.000000
 태양광 발전량(MWh)     0.000000
dtype: float64

In [28]:
_5min.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035067 entries, 0 to 1035066
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   기준일시        1035067 non-null  object 
 1   공급능력(MW)    1035067 non-null  float64
 2   현재수요(MW)    1035067 non-null  float64
 3   최대예측수요(MW)  1035067 non-null  int64  
 4   공급예비력(MW)   1035067 non-null  float64
 5   공급예비율(퍼센트)  1035067 non-null  float64
 6   운영예비력(MW)   1035067 non-null  float64
 7   운영예비율(퍼센트)  1035067 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 63.2+ MB


In [31]:
solar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918000 entries, 0 to 917999
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   거래일자            918000 non-null  object
 1   거래시간            918000 non-null  int64 
 2   지역              918000 non-null  object
 3    태양광 발전량(MWh)   918000 non-null  object
 4    풍력 발전량(MWh)    594000 non-null  object
dtypes: int64(1), object(4)
memory usage: 35.0+ MB


In [40]:
solar[' 태양광 발전량(MWh) ']
# solar.columns

0             
1             
2             
3             
4             
          ... 
917995    3.59
917996    3.46
917997    2.71
917998    0.82
917999    0.18
Name:  태양광 발전량(MWh) , Length: 918000, dtype: object

In [43]:
solar

,거래일자,거래시간,지역,태양광 발전량(MWh),풍력 발전량(MWh)
0,2017-01-01,1,경기도,,0.01
1,2017-01-01,2,경기도,,
2,2017-01-01,3,경기도,,
3,2017-01-01,4,경기도,,0.01
4,2017-01-01,5,경기도,,0.02
...,...,...,...,...,...
917995,2023-02-28,20,부산시,3.59,
917996,2023-02-28,21,부산시,3.46,
917997,2023-02-28,22,부산시,2.71,
917998,2023-02-28,23,부산시,0.82,


https://apihub.kma.go.kr/
<br>예특보

https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl.php?authKey=yEtC0d2xQmKLQtHdsWJi5g

In [69]:
domain = 'https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl.php?'
authKey = 'authKey=yEtC0d2xQmKLQtHdsWJi5g'

url = domain + authKey

with urlopen(url) as f:
    html = f.read()
    print(html)
    

b'#START7777\n# REG_ID TM_FC        TM_EF        MOD NE STN C MAN_ID       MAN_FC     W1 T W2  TA  ST SKY  PREP WF\n11A00101 202306071100 202306071200 A02  0 109 2 she          \xb0\xad\xc7\xfd\xb9\xcc     SE 1 S   21  20 DB03    0 "\xb1\xb8\xb8\xa7\xb8\xb9\xc0\xbd"\n11A00101 202306071100 202306080000 A02  1 109 2 she          \xb0\xad\xc7\xfd\xb9\xcc     SE 1 S   16  60 DB03    1 "\xb1\xb8\xb8\xa7\xb8\xb9\xb0\xed \xc7\xd1\xb6\xa7 \xba\xf1"\n11A00101 202306071100 202306081200 A02  2 109 2 she          \xb0\xad\xc7\xfd\xb9\xcc     SE 1 S   20  60 DB03    1 "\xb1\xb8\xb8\xa7\xb8\xb9\xb0\xed \xc7\xd1\xb6\xa7 \xba\xf1"\n11A00101 202306071100 202306090000 A02  3 109 2 she          \xb0\xad\xc7\xfd\xb9\xcc     E  1 SE  14   0 DB01    0 "\xb8\xbc\xc0\xbd"\n11A00101 202306071100 202306091200 A02  4 109 2 she          \xb0\xad\xc7\xfd\xb9\xcc     SE 1 S   21   0 DB01    0 "\xb8\xbc\xc0\xbd"\n11A00101 202306071100 202306100000 A02  5 109 2 she          \xb0\xad\xc7\xfd\xb9\xcc     SE 1 S   15   

In [73]:
import requests

url = 'https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl.php?'
params ={'authKey' : 'authKey=yEtC0d2xQmKLQtHdsWJi5g'}

response = requests.get(url, params=params)
print(response.content)

b'{\n  "result" : {\n    "status" : 401,\n    "message" : "\xec\x9c\xa0\xed\x9a\xa8\xed\x95\x9c \xec\x9d\xb8\xec\xa6\x9d\xed\x82\xa4\xea\xb0\x80 \xec\x95\x84\xeb\x8b\x99\xeb\x8b\x88\xeb\x8b\xa4."\n  }\n}'


In [ ]:
    serviceKey = "서비스키" # 공공데이터 포털에서 생성된 본인의 서비스 키를 복사 / 붙여넣기
    serviceKeyDecoded = unquote(serviceKey, 'UTF-8') # 공데이터 포털에서 제공하는 서비스키는 이미 인코딩된 상태이므로, 디코딩하여 사용해야 함

    now = datetime.now()
    today = datetime.today().strftime("%Y%m%d")
    y = date.today() - timedelta(days=1)
    yesterday = y.strftime("%Y%m%d")
    nx = 60 # 위도와 경도를 x,y좌표로 변경
    ny = 127
    
        if now.minute<45: # base_time와 base_date 구하는 함수
        if now.hour==0:
            base_time = "2330"
            base_date = yesterday
        else:
            pre_hour = now.hour-1
            if pre_hour<10:
                base_time = "0" + str(pre_hour) + "30"
            else:
                base_time = str(pre_hour) + "30"
            base_date = today
    else:
        if now.hour < 10:
            base_time = "0" + str(now.hour) + "30"
        else:
            base_time = str(now.hour) + "30"
        base_date = today

In [72]:
f.request

AttributeError: 'HTTPResponse' object has no attribute 'request'